In [2]:
import sounddevice as sd
from scipy.io.wavfile import write
import io
from google.cloud import speech
from google.oauth2 import service_account

# === BƯỚC 1: GHI ÂM GIỌNG NÓI ===
fs = 48000  # Tần số mẫu
seconds = 5 # Thời lượng ghi âm (giây)

print(" Đang ghi âm... Nói đi bạn!")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()  # Đợi ghi âm xong
write("thu_am.wav", fs, recording)
print("Ghi âm xong, đã lưu thành 'thu_am.wav'")

# === BƯỚC 2: CẤU HÌNH GOOGLE CLOUD SPEECH-TO-TEXT ===
client_file = "C:/Users/ACER/Downloads/ggspeech-456316-7b6f651a67a9.json"
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

# Đọc file âm thanh
with io.open("thu_am.wav", "rb") as f:
    content = f.read()

audio = speech.RecognitionAudio(content=content)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=48000,
    language_code="vi-VN"
)

# === BƯỚC 3: GỬI YÊU CẦU VÀ IN KẾT QUẢ ===
print(" Đang nhận diện giọng nói...")
response = client.recognize(config=config, audio=audio)

print("Văn bản được nhận diện:")
for result in response.results:
    print(result.alternatives[0].transcript)



 Đang ghi âm... Nói đi bạn!
Ghi âm xong, đã lưu thành 'thu_am.wav'
 Đang nhận diện giọng nói...
Văn bản được nhận diện:
tôi nuôi mèo


In [4]:
for result in response.results:
    recognized_text = result.alternatives[0].transcript
    print("Văn bản nhận diện:", recognized_text)

Văn bản nhận diện: tôi nuôi mèo


In [ ]:
import nltk
import string
import unicodedata
import underthesea
nltk.download('punkt')

In [ ]:
punctuations = set(string.punctuation)

In [ ]:
word = recognized_text

In [ ]:
# Chuẩn hóa Unicode
normalized_text = unicodedata.normalize("NFKC", word)
# Chuyển về chữ thường
lower_text = normalized_text.lower()
print(lower_text)

In [ ]:
underthesea_tokens = underthesea.word_tokenize(lower_text)

In [ ]:
underthesea_punctuations_filtered = [token for token in underthesea_tokens if token not in punctuations]
print(len(underthesea_tokens))
print(len(underthesea_punctuations_filtered))
print(underthesea_punctuations_filtered)

In [ ]:
# Danh sách stopwords tiếng Việt
stopwords = ['và', 'là', 'của', 'có', 'trong', 'những', 'đã', 'này', 'đó', 'thì', 'ra', 'khi', 'vì', 'tôi', 'đang']

# Loại bỏ stopword
filtered_words = [underthesea_punctuations_filtered for underthesea_punctuations_filtered in underthesea_punctuations_filtered if underthesea_punctuations_filtered not in stopwords]

print("Kết quả sau khi loại bỏ stopword:", filtered_words)


In [ ]:


from moviepy.editor import VideoFileClip, concatenate_videoclips
import os

def text_to_video(words_list, video_folder, output_video="output11_video.mp4"):
    # Danh sách để chứa các video clip
    clips = []

    # Duyệt qua từng từ trong danh sách và ánh xạ đến video
    for word in words_list:
        video_file = os.path.join(video_folder, f"{word.lower()}.mp4")
        if os.path.exists(video_file):
            clip = VideoFileClip(video_file)
            clips.append(clip)
        else:
            print(f"Không tìm thấy video cho từ: {word}")

    # Kết hợp các video lại với nhau
    if clips:
        final_clip = concatenate_videoclips(clips, method="compose")
        final_clip.write_videofile(output_video, fps=24)
    else:
        print("Không có video để tạo thành video cuối.")

video_folder = "D:\\firefly\\project_folder\\clips"
  # Đường dẫn tới thư mục chứa video
words_list = filtered_words  # Danh sách các từ bạn muốn ánh xạ tới video
text_to_video(words_list, video_folder)
